In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from pathlib import Path


# const
INPUT_DIR = Path("/kaggle/input/MABe-mouse-behavior-detection")
TRAIN_TRACKING_DIR = INPUT_DIR / "train_tracking"
TRAIN_ANNOTATION_DIR = INPUT_DIR / "train_annotation"
TEST_TRACKING_DIR = INPUT_DIR / "test_tracking"

WORKING_DIR = Path("/kaggle/working")

INDEX_COLS = [
    "video_id",
    "agent_mouse_id",
    "target_mouse_id",
    "video_frame",
]

# taken from others' notebooks

SELF_BEHAVIORS = [
    "biteobject",
    "climb",
    "dig",
    "exploreobject",
    "freeze",
    "genitalgroom",
    "huddle",
    "rear",
    "rest",
    "run",
    "selfgroom",
]

PAIR_BEHAVIORS = [
    "allogroom",
    "approach",
    "attack",
    "attemptmount",
    "avoid",
    "chase",
    "chaseattack",
    "defend",
    "disengage",
    "dominance",
    "dominancegroom",
    "dominancemount",
    "ejaculate",
    "escape",
    "flinch",
    "follow",
    "intromit",
    "mount",
    "reciprocalsniff",
    "shepherd",
    "sniff",
    "sniffbody",
    "sniffface",
    "sniffgenital",
    "submit",
    "tussle",
]

BODY_PARTS = [
    "ear_left",
    "ear_right",
    "nose",
    #"neck",
    "body_center",
    "lateral_left",
    "lateral_right",
    "hip_left",
    "hip_right",
    "tail_base",
    "tail_tip",
]

self_idx = {
    (1,1): 0,
    (2,2): 1,
    (3,3): 2,
    (4,4): 3
}

pair_idx = {
    (1,2): 4,
    (1,3): 5,
    (1,4): 6,
    (2,1): 7,
    (2,3): 8,
    (2,4): 9,
    (3,1): 10,
    (3,2): 11,
    (3,4): 12,
    (4,1): 13,
    (4,2): 14,
    (4,3): 15
}

BEHAVIOR_TO_ID = {
    # Self behaviors (0–10)
    "biteobject": 1,
    "climb": 2,
    "dig": 3,
    "exploreobject": 4,
    "freeze": 5,
    "genitalgroom": 6,
    "huddle": 7,
    "rear": 8,
    "rest": 9,
    "run": 10,
    "selfgroom": 11,

    # Pair behaviors (11–36)
    "allogroom": 1,
    "approach": 2,
    "attack": 3,
    "attemptmount": 4,
    "avoid": 5,
    "chase": 6,
    "chaseattack": 7,
    "defend": 8,
    "disengage": 9,
    "dominance": 10,
    "dominancegroom": 11,
    "dominancemount": 12,
    "ejaculate": 13,
    "escape": 14,
    "flinch": 15,
    "follow": 16,
    "intromit": 17,
    "mount": 18,
    "reciprocalsniff": 19,
    "shepherd": 20,
    "sniff": 21,
    "sniffbody": 22,
    "sniffface": 23,
    "sniffgenital": 24,
    "submit": 25,
    "tussle": 26,
}

In [3]:
train_meta = pd.read_csv(INPUT_DIR/"train.csv")

what do we do with different labs? train them together?

In [9]:
# load data?
train_behavior = train_meta[train_meta["behaviors_labeled"].notna()].copy() # drop videos with no annotation
true_train_meta = train_behavior.copy()
train_behavior.loc[:, "behaviors_labeled_list"] = (
    train_behavior["behaviors_labeled"].apply(eval)
)
# explode the behaviors into rows
train_behavior = train_behavior[["lab_id", "video_id", "behaviors_labeled_list"]]
train_behavior = (
    train_behavior.explode("behaviors_labeled_list")
    .rename(columns={"behaviors_labeled_list": "behaviors_labeled_element"})
)

split_elements = train_behavior["behaviors_labeled_element"].str.split(",", expand=True)
train_behavior["agent"] = split_elements[0].str.replace("'", "", regex=False)
train_behavior["target"] = split_elements[1].str.replace("'", "", regex=False)
train_behavior["behavior"] = split_elements[2].str.replace("'", "", regex=False)

train_behavior = train_behavior[["lab_id", "video_id", "agent", "target", "behavior"]].copy()

true_train_meta.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [8]:
# 4. Filter for self and pair behaviors
# should I use this
train_self_behavior_dataframe = (
    train_behavior[train_behavior["behavior"].isin(SELF_BEHAVIORS)]
    .copy()
)
train_pair_behavior_dataframe = (
    train_behavior[train_behavior["behavior"].isin(PAIR_BEHAVIORS)]
    .copy()
)
print(train_behavior.info())
train_behavior = train_behavior[train_behavior["behavior"].isin(SELF_BEHAVIORS + PAIR_BEHAVIORS)].copy()

print(train_behavior.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4919 entries, 0 to 8788
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lab_id    4919 non-null   object
 1   video_id  4919 non-null   int64 
 2   agent     4919 non-null   object
 3   target    4919 non-null   object
 4   behavior  4919 non-null   object
dtypes: int64(1), object(4)
memory usage: 230.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 4919 entries, 0 to 8788
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lab_id    4919 non-null   object
 1   video_id  4919 non-null   int64 
 2   agent     4919 non-null   object
 3   target    4919 non-null   object
 4   behavior  4919 non-null   object
dtypes: int64(1), object(4)
memory usage: 230.6+ KB
None


In [26]:
def make_features_labels(tracking_path, anno_path):
    try:
        tracking = pd.read_parquet(tracking_path)
    except FileNotFoundError:
        return
    track = track[track["bodypart"].isin(BODY_PARTS)].copy() # when to copy?

    track_pivot = track.pivot_table(
        index=["video_frame", "mouse_id"],
        columns="bodypart",
        values=["x", "y"]
    ).reset_index()

    track_pivot = track_pivot.interpolate(method='linear') # I hate data processing
    track_pivot = track_pivot.ffill().bfill()

    track_pivot = track_pivot.sort_index(axis=1)


    track_wide = track_pivot.pivot_table(
        index="video_frame",
        columns="mouse_id",
        values= track_pivot.columns.drop(["video_frame", "mouse_id"])
    )
    
    # there might be less than 4 mouses tracked, read from metadata to find the number of mouses for each video
    track_wide = track_wide.interpolate(method='linear') # I hate data processing
    track_wide = track_wide.ffill().bfill()
    
    track_wide = track_wide.sort_index(axis=1)  # tidy order


    # CREATE LABELS FROM HERE
    # explode labels

    slot_names = []
    self_idx = {}
    pair_idx = {}
    
    slot_id = 0
    
    # self slots
    for i in range(1, 5):
        self_idx[(i, i)] = slot_id
        slot_names.append(f"self{i}")
        slot_id += 1
    
    # pair slots (ordered)
    for i in range(1, 5):
        for j in range(1, 5):
            if i != j:
                pair_idx[(i, j)] = slot_id
                slot_names.append(f"pair{i}{j}")
                slot_id += 1
    
    assert slot_id == 16
    
    frames = track_wide.index.to_numpy()
    
    label_df = pd.DataFrame(
        0,
        index=frames,
        columns=slot_names,
        dtype=np.int64
    )
    
    label_df.index.name = "video_frame"
    
    anno = pd.read_parquet(anno_path)
    
    for _, row in anno.iterrows():
        agent  = int(row["agent_id"])
        target = int(row["target_id"])
        beh    = row["action"]
    
        beh_id = BEHAVIOR_TO_ID[beh]
    
        if agent == target:
            col = slot_names[self_idx[(agent, agent)]]
        else:
            col = slot_names[pair_idx[(agent, target)]]
    
        label_df.loc[row["start_frame"]:row["stop_frame"], col] = beh_id

        
    return track_wide, label_df


In [9]:
# def make_labels(anno_path):   
#     anno = pd.read_parquet("/kaggle/input/MABe-mouse-behavior-detection/train_annotation/AdaptableSnail/1212811043.parquet")
#     anno.head()
#     frame_labels = []
#     for _, row in anno.iterrows():
#         for frame in range(row["start_frame"], row["stop_frame"] + 1):
#             frame_labels.append({
#                 "frame": frame,
#                 "agent_id" : row["agent_id"],
#                 "target_id" : int(row["target_id"]),
#                 "action" : row["action"]
#             })
#     frame_labels_df = pd.DataFrame(frame_labels)
#     frame_labels_df.head()

# # this for loop probably won't be good

# #problem: target_id for self? it's the same as agent_id

# # annos = pd.read_parquet("/kaggle/input/MABe-mouse-behavior-detection/train_annotation/GroovyShrew/991578442.parquet")
# # annos = annos[annos["target_id"] == annos["agent_id"]]
# # annos.head()

# num_frames = len(df_wide) # how to really find num_frames (non-action at the end, will break Y)

# Y = np.zeros((num_frames + 1, 16), dtype=int) 

# for _, row in frame_labels_df.iterrows():
#     f = row["frame"]
#     i = row["agent_id"]
#     r = row["target_id"]
#     beh = row["action"]

#     beh_id = BEHAVIOR_TO_ID[beh]

#     if i == r:  # self action
#         idx = self_idx[(i,i)]
#     else:
#         idx = pair_idx[(i,r)]

#     Y[f, idx] = beh_id

# print(Y[2])

[0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0]


In [23]:
from tqdm import tqdm
def process_video(row):
    """Process a single video"""
    lab_id = row["lab_id"]
    if lab_id.startswith('MABe22'): 
        return
    video_id = row["video_id"]

    tracking_path = TRAIN_TRACKING_DIR / f"{lab_id}/{video_id}.parquet"
    anno_path = TRAIN_ANNOTATION_DIR / f"{lab_id}/{video_id}.parquet"


    # self_features = make_self_features(metadata=row, tracking=tracking)
    # pair_features = make_pair_features(metadata=row, tracking=tracking)

    # self_features.write_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet")
    # pair_features.write_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet")

    features, labels = make_features(lab_id, video_id)
    features.to_parquet(WORKING_DIR / "features" / f"{video_id}.parquet")
    labels.to_parquet(WORKING_DIR / "labels" / f"{video_id}.parquet")

    return video_id

rows = true_train_meta
# this ain't good
for _, row in tqdm(rows.iterrows(), total=len(rows), desc="Processing videos"):
    process_video(row)


Processing videos: 100%|██████████| 848/848 [04:38<00:00,  3.04it/s] 


In [24]:
test = pd.read_parquet("/kaggle/working/1059582964.parquet")
test.head()

x                                                  \
bodypart    body_center                ear_left               ear_right   
mouse_id              1           2           1           2           1   
video_frame                                                               
0            256.928375  259.561096  254.337341  265.714478  257.576691   
1            256.928375  259.561096  254.454559  265.774231  257.576691   
2            256.928375  259.561096  254.419815  265.755035  257.576691   
3            256.928375  259.561096  254.435913  265.865112  257.576691   
4            256.928375  259.561096  254.559723  265.840881  257.576691   

                                                                         \
bodypart                       nose               tail_base               
mouse_id              2           1           2           1           2   
video_frame                                                               
0            253.268875  265.797089  274.311096  266.671265  261.547028   
1            253.268875  265.726105  274.311096  266.671265  261.512115   
2            253.268875  265.820007  274.315704  266.671265  261.535095   
3            253.268875  265.746063  274.320282  266.671265  261.486542   
4            253.268875  265.737518  274.324890  266.671265  261.496674   

                      y                                                  \
bodypart    body_center                ear_left               ear_right   
mouse_id              1           2           1           2           1   
video_frame                                                               
0            233.626511  233.566818  240.892166  240.928375  241.284576   
1            233.626511  233.566818  240.913528  240.866180  241.284576   
2            233.626511  233.566818  240.836395  240.740402  241.284576   
3            233.626511  233.566818  241.016708  240.788574  241.284576   
4            233.626511  233.566818  240.969833  240.811935  241.284576   

                                                                         
bodypart                       nose               tail_base              
mouse_id              2           1           2           1           2  
video_frame                                                              
0            242.128891  233.380829  242.796249  254.485504  218.579285  
1            242.128891  233.339371  242.796249  254.485504  218.556870  
2            242.128891  233.385681  242.786972  254.485504  218.520996  
3            242.128891  233.388962  242.777695  254.485504  218.499298  
4            242.128891  233.391556  242.768417  254.485504  218.499451

In [7]:
def make_windows(, seq_len)
X = track_wide.to_numpy()  # shape (num_frames, feature_dim)
X.shape

seq_len = 40
stride = 20

X_sequences = []
for start in range(0, len(df_wide) - seq_len + 1, stride):
    seq = df_wide.iloc[start : start + seq_len].values
    X_sequences.append(seq)

X = np.array(X_sequences)
print(X.shape)  # (num_sequences, 40, num_features)
nan_count_X = np.isnan(X).sum()
print(nan_count_X)

(4497, 40, 64)
0


,frame,agent_id,target_id,action
0,2,1,3,chase
1,3,1,3,chase
2,4,1,3,chase
3,5,1,3,chase
4,6,1,3,chase


In [10]:
# with this you need to do something so Y size is the same as X
def build_Y_sequences(Y, seq_len=40, stride=20):
    """
    Y: np.array of shape (num_frames, 16)
    Returns: np.array of shape (num_sequences, seq_len, 16)
    """
    Y_sequences = []

    for start in range(0, len(Y) - seq_len + 1, stride):
        seq = Y[start : start + seq_len]
        Y_sequences.append(seq)

    return np.array(Y_sequences)


Y_seq = build_Y_sequences(Y, seq_len=40, stride=20)
print(Y_seq.shape)

(4497, 40, 16)


In [27]:
#training
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, hamming_loss
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter

import tensorflow as tf

dropout = 0.3 #dropout
n_splits = 3 #for fold
batch_size = 64

# sizes for LSTM
seq_len = X.shape[1]
num_features = X.shape[2] 
num_labels = Y_seq.shape[2]  # 16

num_classes_per_output = [
    11, 11, 11, 11,  # self-actions for 4 mice 
    26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26  # pair-actions for 12 pairs
]

def focal_loss(gamma=2., alpha=4.0):
    def focal_loss_fixed(y_true, y_pred):
        # Sparse Categorical Focal Loss
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=y_pred.shape[-1])
        y_true = tf.cast(y_true, tf.float32)
        
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        
        # Calculate Cross Entropy
        cross_entropy = -y_true * tf.math.log(y_pred)
        
        # Calculate Focal Loss weights
        loss = alpha * tf.math.pow(1 - y_pred, gamma) * cross_entropy
        
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))
    return focal_loss_fixed

def build_model(seq_len, num_features):

    # model = Sequential()
    
    # # LSTM layer: return_sequences=True to predict per frame
    # model.add(LSTM(128, input_shape=(seq_len, num_features), return_sequences=True))
    # model.add(Dropout(0.3))
    # model.add(BatchNormalization())

    # # Output layer: 16 behaviors, sigmoid for multi-label
    # model.add(Dense(num_classes, activation='softmax'))
    
    # adam_optimizer = Adam(
    #     learning_rate=0.001,
    #     # Implement Gradient Clipping
    #     # We clip by L2 norm, a common and effective technique for LSTMs.
    #     clipnorm=1.0 
    # )
    
    # # Compile with binary_crossentropy for multi-label
    # model.compile(
    #     optimizer= adam_optimizer,
    #     loss= 'categorical_crossentropy',
    #     metrics=['accuracy']
    # )
    
    inputs = Input(shape=(seq_len, num_features))
    x = LSTM(128, return_sequences=True)(inputs)  # LSTM with sequences

    outputs = []
    for n_classes in num_classes_per_output:
        out = Dense(n_classes, activation='softmax')(x)  # softmax per output
        outputs.append(out)

    adam_optimizer = Adam(
        learning_rate=0.001,
        clipnorm=1.0         # <-- CRITICAL: Prevents gradients from exploding
    )
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer= adam_optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'] * 16
    )
    
    return model

gkf = GroupKFold(n_splits=n_splits) # DON'T USE STRATIFIED K FOLD BUT WHY I FORGOR
# do this later anyway


#Scale the features
N, T, F = X.shape # e.g. (36000, 100, 16)
X_flat = X.reshape(-1, F)
scaler = StandardScaler()
X_flat_scaled = scaler.fit_transform(X_flat)
#Reshape back to 3D sequences
X_scaled = X_flat_scaled.reshape(N, T, F)
print(X_scaled.shape)

# 1. Split the raw arrays first (X_scaled and Y_seq both have N samples)
X_train, X_val, Y_seq_train, Y_seq_val = train_test_split(
    X_scaled, 
    Y_seq, 
    test_size=0.2, 
    random_state=42
)


# # Y_seq_train is the array of shape (N_train, T, 16)

# 1. Flatten Y_seq_train into a single array of frame-labels
# Shape: (N_train * T * 16,)
y_train_flat_all = Y_seq_train.flatten()
print(y_train_flat_all)


# Filter the array to keep only non-zero labels
non_zero_labels = y_train_flat_all[y_train_flat_all != 0]

# Count the frequency of each non-zero label
label_counts = Counter(non_zero_labels)

# Convert to a DataFrame for easy viewing and sorting
import pandas as pd
counts_df = pd.DataFrame(label_counts.items(), columns=['Class ID', 'Count'])
counts_df = counts_df.sort_values(by='Count', ascending=False).reset_index(drop=True)

print("\n--- Frequency of Non-Zero Behaviors ---")
print(counts_df.to_markdown(index=False))



# 2. Identify all unique classes present across the entire dataset (0 up to 25)
# Note: You need to know the highest possible class ID, which is 25.
unique_classes_present = np.unique(y_train_flat_all)

# 3. Compute Class Weights
# 'balanced' mode automatically sets weights inversely proportional to class frequencies.
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=unique_classes_present,
    y=y_train_flat_all
)
MAX_WEIGHT_CAP = 5
# 4. Convert the array back into a dictionary for Keras
class_weights_dict = dict(zip(unique_classes_present, class_weights_array))
for class_id in list(class_weights_dict.keys()): # Iterate over a copy of keys
    weight = class_weights_dict[class_id]
    if weight > MAX_WEIGHT_CAP:
        class_weights_dict[class_id] = MAX_WEIGHT_CAP

print("--- Calculated Class Weights (ID: Weight) ---")
# Displaying the results to show how much more weight is assigned to minority classes
for class_id, weight in list(class_weights_dict.items())[:5]: # Show first 5
    print(f"Class {class_id}: {weight:.2f}")

# Example interpretation: If Class 0 has a weight of ~0.5 and Class 25 has a weight of ~50.0,
# misclassifying Class 25 is 100 times worse than misclassifying Class 0.

(4497, 40, 64)
[0 0 0 ... 0 0 0]

--- Frequency of Non-Zero Behaviors ---
|   Class ID |   Count |
|-----------:|--------:|
|          3 |    9001 |
|          5 |    5580 |
|          6 |    5093 |
|          7 |    1609 |
--- Calculated Class Weights (ID: Weight) ---
Class 0: 0.20
Class 3: 5.00
Class 5: 5.00
Class 6: 5.00
Class 7: 5.00


In [28]:

# # --- Proceed to Step 2 (Applying Weights) ---

# Map the weights to every label in the training set (N_train, T, 16)
sample_weight_matrix = np.vectorize(class_weights_dict.get)(Y_seq_train)

# Convert the sample_weight_matrix into the Keras LIST format
sample_weight_list = [sample_weight_matrix[:, :, i] for i in range(16)]

# 2. NOW convert the splits into the list format your model needs (16 outputs)
y_train = [Y_seq_train[:, :, i] for i in range(16)]
y_val   = [Y_seq_val[:, :, i] for i in range(16)]

model = build_model(seq_len, num_features)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=1,
    batch_size=batch_size,
    sample_weight=sample_weight_list,
    verbose=1
)

57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 142ms/step - dense_48_accuracy: 0.5623 - dense_48_loss: 0.3109 - dense_49_accuracy: 0.5333 - dense_49_loss: 0.3354 - dense_50_accuracy: 0.4400 - dense_50_loss: 0.3537 - dense_51_accuracy: 0.5032 - dense_51_loss: 0.3383 - dense_52_accuracy: 0.3842 - dense_52_loss: 0.9209 - dense_53_accuracy: 0.3497 - dense_53_loss: 0.9244 - dense_54_accuracy: 0.3643 - dense_54_loss: 1.1287 - dense_55_accuracy: 0.3062 - dense_55_loss: 0.5968 - dense_56_accuracy: 0.4778 - dense_56_loss: 0.4755 - dense_57_accuracy: 0.3905 - dense_57_loss: 0.5047 - dense_58_accuracy: 0.3388 - dense_58_loss: 0.5810 - dense_59_accuracy: 0.4058 - dense_59_loss: 0.5137 - dense_60_accuracy: 0.3856 - dense_60_loss: 0.5638 - dense_61_accuracy: 0.4275 - dense_61_loss: 0.7251 - dense_62_accuracy: 0.4696 - dense_62_loss: 0.5413 - dense_63_accuracy: 0.3381 - dense_63_loss: 0.6292 - loss: 9.4455 - val_dense_48_accuracy: 1.0000 - val_dense_48_loss: 0.0633 - val_dense_49_accuracy: 1.0000 - val_dense_49_loss

In [30]:
import numpy as np
import pandas as pd

# Assume X_val (validation inputs) is available

# 1. Select the first sequence for inspection
X_sample = X_val[0:1] # Shape (1, seq_len, num_features)

# 2. Get predictions (y_pred_list is a list of 16 arrays (16 neurons output, 4 + 12))
# Each array in the list has shape: (1, seq_len, num_classes)
y_pred_list = model.predict(X_sample)

for i, pred in enumerate(y_pred_list):
    print(f"Output {i}: shape = {pred.shape}")
    print(pred)

# 3. Choose a specific time step (frame) to inspect
FRAME_INDEX = 3 

print(f"--- Predictions for Sequence 0, Frame {FRAME_INDEX} ---")

output_data = []

# Loop through each of the 16 behavioral outputs
for i, pred_array in enumerate(y_pred_list):
    
    # Extract the prediction vector for the chosen frame
    # pred_vector shape: (num_classes,)
    pred_vector = pred_array[0, FRAME_INDEX, :] 
    
    # Get the class with the highest probability
    predicted_class_id = np.argmax(pred_vector)
    max_probability = np.max(pred_vector)
    
    # Get the name/class count for context
    output_name = f"Output {i+1} ({'Self' if i < 4 else 'Pair'})"
    n_classes = len(pred_vector)

    output_data.append({
        'Output Name': output_name,
        'Classes': n_classes,
        'Predicted ID': predicted_class_id,
        'Max Probability': f"{max_probability:.4f}",
        'Prediction Vector (Example)': pred_vector[:min(n_classes, 5)] # Show first 5 probabilities
    })

# 4. Display the results in a formatted table
pred_df = pd.DataFrame(output_data)
print(pred_df.to_markdown(index=False))

print("\n\n--- Raw Prediction Vector Examples ---")
print("These are the probabilities (softmax output) for the first few classes in each output.")
print("The full vector length is either 11 or 26.")

# Print the full vectors for the first two outputs as an example
print(f"\nOutput 1 (11 classes) Vector:\n{y_pred_list[0][0, FRAME_INDEX, :]}")
print(f"\nOutput 5 (26 classes) Vector:\n{y_pred_list[4][0, FRAME_INDEX, :]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Output 0: shape = (1, 40, 11)
[[[1.82278782e-01 7.07856193e-02 6.35230988e-02 9.00594518e-02
   6.87034205e-02 8.88280496e-02 8.24742690e-02 8.00233781e-02
   9.34133083e-02 8.44425261e-02 9.54679996e-02]
  [3.63122642e-01 4.84428853e-02 3.90781350e-02 7.99590573e-02
   4.53748778e-02 6.81126565e-02 7.17513785e-02 5.70326522e-02
   8.07887167e-02 6.64324388e-02 7.99045339e-02]
  [6.56614482e-01 2.29238104e-02 1.70496628e-02 5.08041382e-02
   2.13870760e-02 3.27332430e-02 4.92553376e-02 2.72350870e-02
   4.45299596e-02 3.49001959e-02 4.25669327e-02]
  [8.93769860e-01 5.91998687e-03 4.32328694e-03 1.84681676e-02
   5.89036150e-03 8.44706967e-03 2.16430500e-02 7.39216432e-03
   1.20666865e-02 1.00716958e-02 1.20076425e-02]
  [9.76169109e-01 1.03294791e-03 8.07524833e-04 4.54738783e-03
   1.14557706e-03 1.52524456e-03 6.89159753e-03 1.44824514e-03
   2.08997959e-03 2.03966210e-03 2.30271020e-03]
  [9.94047165e-01 2.04169948e-04 1.81895652e-04 1.1677520

In [30]:

# Predict on validation set
# y_pred is already a LIST of 16 NumPy arrays (output of model.predict on multi-output model)
y_pred = model.predict(X_val)

print(y_pred.size)

# --- FIX START ---

# 1. Convert y_val (list of 16 arrays of shape (N_val, seq_len)) back into a single array:
# The original shape was (N_val, seq_len, 16)
# np.stack joins them along a new axis (the last one, axis=-1)
Y_val_stacked = np.stack(y_val, axis=-1)

# 2. Flatten for frame-level metrics:
# (N_val, seq_len, 16) -> (N_val * seq_len, 16)
y_val_flat = Y_val_stacked.reshape(-1, Y_val_stacked.shape[-1]) # or just (-1, 16)


# 3. Handle y_pred: y_pred is also a list of 16 arrays. Stack and flatten it too.
Y_pred_stacked = np.stack(y_pred, axis=-1)
y_pred_flat = Y_pred_stacked.reshape(-1, Y_pred_stacked.shape[-1])

# --- FIX END ---

print(y_pred_flat)
y_val_df = pd.DataFrame(y_pred_flat)
non_zero_rows = y_val_df[(y_val_df != 0).any(axis=1)]

# The check for 'is_binary' needs to be done on the true labels (y_val_flat)
# if y_val contained integer labels, which it should based on your loss function.
is_binary = np.isin(y_val_flat, [0, 1]).all() 

print(is_binary)

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


AttributeError: 'list' object has no attribute 'size'

In [31]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

# --- Define the class structure for interpretation ---
# The first 4 have 11 classes, the next 12 have 26 classes
NUM_CLASSES_PER_OUTPUT = [11, 11, 11, 11, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]
OUTPUT_NAMES = [
    'Mouse 1 Self', 'Mouse 2 Self', 'Mouse 3 Self', 'Mouse 4 Self',
    'Pair 1', 'Pair 2', 'Pair 3', 'Pair 4', 'Pair 5', 'Pair 6', 
    'Pair 7', 'Pair 8', 'Pair 9', 'Pair 10', 'Pair 11', 'Pair 12'
]

# 1. Predict on validation set
y_pred_list = model.predict(X_val)
# y_val is already a list of 16 true label arrays (integer labels)

# --- Loop through each of the 16 outputs ---
print("\n--- Detailed Per-Output Validation Metrics ---")
print("-" * 50)

summary_data = []

for i in range(len(OUTPUT_NAMES)):
    output_name = OUTPUT_NAMES[i]
    n_classes = NUM_CLASSES_PER_OUTPUT[i]

    # A. Extract True Labels and Predictions for this specific output (i)
    # y_val[i] is shape (N_val, seq_len) - True integer labels
    # y_pred_list[i] is shape (N_val, seq_len, n_classes) - Predicted probabilities

    y_true_sequence = y_val[i]
    y_pred_prob_sequence = y_pred_list[i]
    
    # B. Flatten the sequences to get frame-level metrics (N_frames,)
    # True labels are flattened directly
    y_true_flat = y_true_sequence.flatten()
    
    # Predictions need argmax after flattening the N_val, seq_len dimensions
    y_pred_prob_flat = y_pred_prob_sequence.reshape(-1, n_classes)
    y_pred_flat = np.argmax(y_pred_prob_flat, axis=-1)

    # C. Calculate Metrics
    
    # Accuracy
    acc = accuracy_score(y_true_flat, y_pred_flat)
    
    # Macro F1-Score (Crucial for class imbalance)
    macro_f1 = f1_score(y_true_flat, y_pred_flat, average='macro', zero_division=0)
    
    # Store data for summary table
    summary_data.append({
        'Output': output_name,
        'Classes': n_classes,
        'Accuracy': f"{acc:.4f}",
        'Macro F1': f"{macro_f1:.4f}"
    })
    
    # D. Print Detailed Report
    print(f"## {output_name} ({n_classes} Classes)")
    print(f"Frame Accuracy: {acc:.4f} | Macro F1: {macro_f1:.4f}")
    
    # Generate report, limiting classes to the actual number for this output (0 to n_classes-1)
    report = classification_report(
        y_true_flat, 
        y_pred_flat, 
        labels=np.arange(n_classes), # Only show classes that exist in this output
        zero_division=0,
        output_dict=True
    )
    
    # Print specific metrics for a high-level view (e.g., F1 for classes 0, 1, 2)
    print("Top 3 Class F1-Scores:")
    for class_id in range(min(n_classes, 3)):
        f1 = report[str(class_id)]['f1-score']
        print(f"  Class {class_id}: {f1:.4f}")
        
    print("-" * 50)

# --- Print Summary Table ---
print("\n\n### Overall Validation Summary")
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_markdown(index=False))

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step

--- Detailed Per-Output Validation Metrics ---
--------------------------------------------------
## Mouse 1 Self (11 Classes)
Frame Accuracy: 0.9999 | Macro F1: 0.3333
Top 3 Class F1-Scores:
  Class 0: 0.9999
  Class 1: 0.0000
  Class 2: 0.0000
--------------------------------------------------
## Mouse 2 Self (11 Classes)
Frame Accuracy: 1.0000 | Macro F1: 1.0000
Top 3 Class F1-Scores:
  Class 0: 1.0000
  Class 1: 0.0000
  Class 2: 0.0000
--------------------------------------------------
## Mouse 3 Self (11 Classes)
Frame Accuracy: 1.0000 | Macro F1: 1.0000
Top 3 Class F1-Scores:
  Class 0: 1.0000
  Class 1: 0.0000
  Class 2: 0.0000
--------------------------------------------------
## Mouse 4 Self (11 Classes)
Frame Accuracy: 1.0000 | Macro F1: 0.5000
Top 3 Class F1-Scores:
  Class 0: 1.0000
  Class 1: 0.0000
  Class 2: 0.0000
--------------------------------------------------
## Pair 1 (26 Classes)
Frame Accuracy: 0.9640 | Macro F1: 0.1963
